In [1]:
import numpy as np
import scipy as sp
from functions.propagators import TwoBody
from functions.read_ICs import read_ICs
from functions.density_funcs import (
    alt_density_1d,
    eval_GMM_PDF,
    integrate_cdf,
)

prop = TwoBody(mu=3.986e5)
# set ICs
r0 = np.array([100, 200, 500, 1000, 10000, 25000]) + 6371
r1 = 6371 + 35786
sma = (r0 + r1) / 2
v0 = np.sqrt(2 * prop.mu / r0 - prop.mu / sma)
tf = 2 * np.pi * np.sqrt(sma**3 / prop.mu)
zro = 0 * r0
ics = np.array(
    [
        r0,
        zro,
        zro,
        zro,
        v0,
        zro,
    ]
).T
ids = [100, 200, 500, 1000, 10000, 25000]
GTO_ICs = ics
states = []
times = []

for idx in range(len(ids)):
    prop.propagate(GTO_ICs[idx, :], tf[idx])
    states.append(prop.states)
    times.append(prop.ts)

In [2]:
ps, alts, cdf, xcdf = alt_density_1d(states[2], times[2], tf[2], 0)

# Case 1: MEO Linear

In [3]:
  
alt_block = (20000 + 6371, 30000 + 6371)

indices = (alts >= alt_block[0]) * (alts < alt_block[1])
np.trapezoid(ps[indices], alts[indices])
    

np.float64(0.2592892999999999)

# Case 2: MEO weighted by proximity to 12hr orbit
### Standard deviation of 3000 km (so Galileo is roughly 1sigma)

In [4]:
GPSalt = 26560
cluster_pdf = sp.stats.norm(loc=26560, scale=3000)
normal_pts = cluster_pdf.pdf(alts)
totProb = normal_pts * ps
np.trapezoid(totProb, alts)

np.float64(2.0062209656217194e-05)

# Ingest TLE data

In [5]:
from functions.tle_histogram import SMA_GMM

### GMM with 1 cluster

In [6]:
gmm = SMA_GMM("data/tles.txt", 1)
gmm_pdf = eval_GMM_PDF(gmm, alts)
gmm_prod = gmm_pdf * ps
np.trapezoid(gmm_prod, alts)

np.float64(1.0735063305333758e-05)

### GMM with 2 clusters

In [7]:
gmm = SMA_GMM("data/tles.txt", 2)
gmm_pdf = eval_GMM_PDF(gmm, alts)
gmm_prod = gmm_pdf * ps
np.trapezoid(gmm_prod, alts)

np.float64(2.655721061681149e-05)

In [8]:
cdf_dro = cdf
integrate_cdf(cdf_dro, gmm_pdf)

np.float64(2.7200486889329148e-05)

### GMM with 3 clusters

In [9]:
gmm = SMA_GMM("data/tles.txt", 3)
gmm_pdf = eval_GMM_PDF(gmm, alts)
gmm_prod = gmm_pdf * ps
np.trapezoid(gmm_prod, alts)

np.float64(2.63007195952964e-05)

In [10]:
integrate_cdf(cdf_dro, gmm_pdf)

np.float64(2.6940484363622767e-05)

### GMM with 4 clusters

In [11]:
gmm = SMA_GMM("data/tles.txt", 4)
gmm_pdf = eval_GMM_PDF(gmm, alts)
gmm_prod = gmm_pdf * ps
np.trapezoid(gmm_prod, alts)

np.float64(3.1198905311905745e-05)

In [12]:
integrate_cdf(cdf_dro, gmm_pdf)

np.float64(3.21136474957422e-05)

### GMM with 5 clusters

In [13]:
gmm = SMA_GMM("data/tles.txt", 5)
gmm_pdf = eval_GMM_PDF(gmm, alts)
gmm_prod = gmm_pdf * ps
np.trapezoid(gmm_prod, alts)

np.float64(3.119058276284492e-05)

In [14]:
integrate_cdf(cdf_dro, gmm_pdf)

np.float64(3.210255744370288e-05)